[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_signatures_validate.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Validate the signatures contained in an input document
Extract and validate signature information for all
digital signatures in the input document, then print the
results to the console.

In [ ]:
import io
import os
import hashlib
from pdftools_sdk.pdf import Document, DocumentSignature
from pdftools_sdk.signature_validation import *
from pdftools_sdk.signature_validation.profiles import Default, RevocationCheckPolicy

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceSigned.pdf'
input_file = 'InvoiceSigned.pdf'
prepare_file(input_url, input_file)
cer_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/swisscom-rootca-4.cer'
cer_file = 'swisscom-rootca-4.cer'
prepare_file(cer_url, cer_file)
cert_dir = '.'  # Placeholder directory with swisscom-rootca-4.cer

In [ ]:
def constraint_to_string(indication: Indication, sub_indication: str, message: str):
    # Convert indication to a string based on its value
    indication_str = (
        "" if indication == Indication.VALID else
        "?" if indication == Indication.INDETERMINATE else
        "!"
    )

    # Return the formatted string
    return f"{indication_str}{sub_indication} {message}"

In [ ]:
def format_sha1_digest(fingerprint: str, delimiter: str):
    return delimiter.join(fingerprint[i:i+2] for i in range(0, len(fingerprint), 2))

In [ ]:
def print_certificate(cert: Certificate):
    if cert is not None:
        print(f"    - Subject    : {cert.subject_name}")
        print(f"    - Issuer     : {cert.issuer_name}")
        print(f"    - Validity   : {cert.not_before} - {cert.not_after}")
        try:
            # Convert the list of integers to bytes
            raw_data_bytes = bytes(cert.raw_data)

            # Fingerprint calculation using hashlib
            fingerprint = hashlib.sha1(raw_data_bytes).hexdigest().upper()
            print(f"    - Fingerprint: {format_sha1_digest(fingerprint, '-')}")
        except Exception as ex:
            print(str(ex))
        # Extract and print the individual DataSource names
        sources = [source.name for source in DataSource if source in cert.source]
        print(f"    - Source     : {', '.join(sources)}")
        print(f"    - Validity   : {constraint_to_string(cert.validity.indication, cert.validity.sub_indication.name, cert.validity.message)}")
    else:
        print("    - null")

In [ ]:
def print_signature_content(content: SignatureContent):
    if content is not None:
        print(f"  - Validity  : {constraint_to_string(content.validity.indication, content.validity.sub_indication.name, content.validity.message)}")

        if isinstance(content, UnsupportedSignatureContent):
            pass  # No action for unsupported content
        elif isinstance(content, CmsSignatureContent):
            print(f"  - Validation: {content.validation_time} from {content.validation_time_source.name}")
            print(f"  - Hash      : {content.hash_algorithm.name}")
            print("  - Signing Cert")
            print_certificate(content.signing_certificate)
            print("  - Chain")
            for index, cert in enumerate(content.certificate_chain, start=1):
                print(f"  - Issuer Cert {index}")
                print_certificate(cert)
            print(f"  - Chain     : {'complete' if content.certificate_chain.is_complete else 'incomplete'} chain")
            print("  Time-Stamp")
            print_signature_content(content.time_stamp)
        elif isinstance(content, TimeStampContent):
            print(f"  - Validation: {content.validation_time} from {content.validation_time_source.name}")
            print(f"  - Hash      : {content.hash_algorithm.name}")
            print(f"  - Time      : {content.date}")
            print("  - Signing Cert")
            print_certificate(content.signing_certificate)
            print("  - Chain")
            for index, cert in enumerate(content.certificate_chain, start=1):
                print(f"  - Issuer Cert {index}")
                print_certificate(cert)
            print(f"  - Chain     : {'complete' if content.certificate_chain.is_complete else 'incomplete'} chain")
        else:
            print(f"Unsupported signature content type {str(type(content))}")
    else:
        print("  - null")

In [ ]:
def on_constraint_event(message: str, indication: Indication, sub_indication: SubIndication, signature: DocumentSignature, data_part: str):
    print(f"  - {signature.name}" + (f": {data_part}" if len(data_part) > 0 else "") + ": " +
          constraint_to_string(indication, sub_indication.name, message))

In [ ]:
def validate(input_file: str, cert_dir: str):
    # Use the default validation profile as a base for further settings
    profile = Default()

    # For offline operation, build a custom trust list from the file system and disable external revocation checks
    if cert_dir:
        print("Using 'offline' validation mode with custom trust list.")
        print()

        # create a CustomTrustList to hold the certificates
        ctl = CustomTrustList()

        # Iterate through files in the certificate directory and add certificates to the custom trust list
        if os.path.isdir(cert_dir):
            for file_name in os.listdir(cert_dir):
                try:
                    with io.FileIO(os.path.join(cert_dir, file_name), 'rb') as cert_stream:
                        if file_name.endswith(".cer") or file_name.endswith(".pem"):
                            ctl.add_certificates(cert_stream)
                        elif file_name.endswith(".p12") or file_name.endswith(".pfx"):
                            # If a password is required, use add_archive(certStr, password).
                            ctl.add_archive(cert_stream)
                except Exception as e:
                    print(f"Could not add certificate '{file_name}' to custom trust list: {e}")
        else:
            print(f"Directory {cert_dir} is missing. No certificates were added to the custom trust list.")
        print()

        profile.custom_trust_list = ctl

        # Configure validation options
        validation_options = profile.validation_options
        validation_options.time_source = TimeSource.PROOF_OF_EXISTENCE | TimeSource.EXPIRED_TIME_STAMP | TimeSource.SIGNATURE_TIME
        validation_options.certificate_sources = DataSource.EMBED_IN_SIGNATURE | DataSource.EMBED_IN_DOCUMENT | DataSource.CUSTOM_TRUST_LIST

        # Disable revocation checks.
        profile.signing_cert_trust_constraints.revocation_check_policy = RevocationCheckPolicy.NO_CHECK
        profile.time_stamp_trust_constraints.revocation_check_policy = RevocationCheckPolicy.NO_CHECK

    # Validate ALL signatures in the document (not only the latest)
    signatureSelector = SignatureSelector.ALL

    # Create the validator object and event listeners
    validator = Validator()
    validator.add_constraint_handler(on_constraint_event)

    try:
        with io.FileIO(input_file, 'rb') as in_stream:
            # Open input document
            # If a password is required, use Open(inStr, password)
            with Document.open(in_stream) as document:
                print("Validation Constraints")
                results = validator.validate(document, profile, signatureSelector)
                print()
                print(f"Signatures validated: {len(results)}")
                print()

                for result in results:
                    field = result.signature_field
                    print(f"{field.field_name} of {field.name}")
                    try:
                        print(f"  - Revision  : {'latest' if field.revision.is_latest else 'intermediate'}")
                    except Exception as ex:
                        print(f"Unable to validate document Revision: {str(ex)}")

                    print_signature_content(result.signature_content)
                    print()
    except Exception as e:
        print(f"Unable to validate file: {e}")

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>")

    # Optional: Set your proxy configuration
    # Sdk.set_proxy("http://myproxy:8080")
    
    validate(input_file, cert_dir)

    print(f"Signatures validated successfully")
except Exception as e:
    print(f"An error occurred: {e}")